In [2]:
import glob
import gzip
import os
import shutil
import sys
import time
import warnings
import zipfile
from datetime import date

import cdsapi
import h5py
import numpy
import numpy as np
import pandas as pd
import urllib3
import xarray

warnings.filterwarnings("ignore")
import pycountry

sys.path.append(os.getcwd() + "/../cds-backend/code/")
import copy
import glob
import json

import cds_eua4 as eua
import numba
import pandas
import psutil
import requests
from numba import njit


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


def datetime_to_seconds(dates, ref="1900-01-01T00:00:00"):
    """from datetime64 to seconds since 1900-01-01 00:00:00"""
    return ((dates - np.datetime64(ref)) / np.timedelta64(1, "s")).astype(np.int64)


def seconds_to_datetime(seconds, ref="1900-01-01"):
    """from seconds to datetime64"""
    seconds = np.asarray(seconds)
    return pd.to_datetime(seconds, unit="s", origin=ref)


import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt

matplotlib.rcParams.update({"font.size": 20})
matplotlib.rcParams["figure.figsize"] = (20, 10)
font = {
    "family": "normal",
    # 'weight' : 'bold',
    "size": 22,
}
matplotlib.rc("font", **font)

In [3]:
stations = glob.glob(
    "/scratch/das/federico/MERGED_02APR2023_arctic_giub/*v1.nc"  # 0-20000-0-01107*
)
stations[:10]

['/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-03302_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-16224_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-11679_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-41196_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-10361_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20400-0-5009_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-246-0-101103_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20300-0-93871_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-07002_CEUAS_merged_v1.nc',
 '/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-07093_CEUAS_merged_v1.nc']

In [6]:
bad_stats = []
for stat in stations[:]:
    target_station = glob.glob(
        "/mnt/users/scratch/leo/scratch/converted_v11/long/" + stat.split("/")[-1]
    )[0]

    with eua.CDMDataset(target_station) as file:
        df = file.to_dataframe(
            groups=["observations_table"],
            variables=[
                "date_time",
                "z_coordinate",
                "z_coordinate_type",
                "z_coordinate_method",
                "observed_variable",
                "observation_value",
            ],
        )
    # df["date_time"] = seconds_to_datetime(df.date_time.values)
    df_geo = df[df.observed_variable == 117]
    df_nan = df_geo[np.isnan(df_geo.z_coordinate)]
    if len(df_nan) > 0:
        print(stat)
        print(target_station)
        display(df_nan)
        print()
        bad_stats.append(target_station)
        if len(bad_stats) > 8:
            break

/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-20891_CEUAS_merged_v1.nc
/mnt/users/scratch/leo/scratch/converted_v11/long/0-20000-0-20891_CEUAS_merged_v1.nc


,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
2400136,1952-05-18 07:11:00,NaN,1,-2147483648,117,1961.329956
2400137,1952-05-18 07:11:00,NaN,1,-2147483648,117,4903.325195
2400138,1952-05-18 07:11:00,NaN,1,-2147483648,117,9806.650391
2400139,1952-05-18 07:11:00,NaN,1,-2147483648,117,14709.974609
2400140,1952-05-18 07:11:00,NaN,1,-2147483648,117,19613.300781
...,...,...,...,...,...,...
2581827,1960-12-31 00:23:00,NaN,1,-2147483648,117,2255.529541
2581828,1960-12-31 00:23:00,NaN,1,-2147483648,117,5197.524414
2581848,1960-12-31 06:22:00,NaN,1,-2147483648,117,1274.864502
2581849,1960-12-31 06:22:00,NaN,1,-2147483648,117,2255.529541



/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-34300_CEUAS_merged_v1.nc
/mnt/users/scratch/leo/scratch/converted_v11/long/0-20000-0-34300_CEUAS_merged_v1.nc


,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
4021646,1947-08-28 03:05:00,NaN,1,-2147483648,117,0.000000
4022347,1947-11-24 23:06:00,NaN,1,-2147483648,117,0.000000
4024064,1948-03-23 23:06:00,NaN,1,-2147483648,117,0.000000
4068346,1953-09-24 14:27:00,NaN,1,-2147483648,117,147099.750000
4111245,1955-07-17 14:28:00,NaN,1,-2147483648,117,176519.703125
...,...,...,...,...,...,...
4326780,1960-12-31 00:21:00,NaN,1,-2147483648,117,3334.260986
4326781,1960-12-31 00:21:00,NaN,1,-2147483648,117,6276.255859
4326803,1960-12-31 06:24:00,NaN,1,-2147483648,117,2353.595947
4326804,1960-12-31 06:24:00,NaN,1,-2147483648,117,3334.260986



/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-03171_CEUAS_merged_v1.nc
/mnt/users/scratch/leo/scratch/converted_v11/long/0-20000-0-03171_CEUAS_merged_v1.nc


...

,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
13546309,1941-06-20 03:16:00,NaN,1,-2147483648,117,1961.329956
13546310,1941-06-20 03:16:00,NaN,1,-2147483648,117,4903.325195
13546311,1941-06-20 03:16:00,NaN,1,-2147483648,117,9806.650391
13546312,1941-06-20 03:16:00,NaN,1,-2147483648,117,14709.974609
13546313,1941-06-20 03:16:00,NaN,1,-2147483648,117,19613.300781
...,...,...,...,...,...,...
13763145,1958-04-09 18:35:00,NaN,1,-2147483648,117,1765.197021
13763146,1958-04-09 18:35:00,NaN,1,-2147483648,117,4707.191895
13763193,1958-04-10 00:31:00,NaN,1,-2147483648,117,784.531982
13763194,1958-04-10 00:31:00,NaN,1,-2147483648,117,1765.197021



/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-23205_CEUAS_merged_v1.nc
/mnt/users/scratch/leo/scratch/converted_v11/long/0-20000-0-23205_CEUAS_merged_v1.nc


,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
20109466,1949-01-21 02:12:00,NaN,1,-2147483648,117,39226.601562
20137675,1952-12-10 02:21:00,NaN,1,-2147483648,117,88259.851562
20137787,1952-12-13 14:11:00,NaN,1,-2147483648,117,88259.851562
20138013,1952-12-21 14:15:00,NaN,1,-2147483648,117,88259.851562
20138116,1952-12-25 02:13:00,NaN,1,-2147483648,117,88259.851562
...,...,...,...,...,...,...
20292184,1960-12-13 00:05:00,NaN,1,-2147483648,117,2059.396484
20292185,1960-12-13 00:05:00,NaN,1,-2147483648,117,5001.391602
20292196,1960-12-13 18:04:00,NaN,1,-2147483648,117,2059.396484
20292197,1960-12-13 18:04:00,NaN,1,-2147483648,117,5001.391602



/scratch/das/federico/MERGED_02APR2023_arctic_giub/0-20000-0-27612_CEUAS_merged_v1.nc
/mnt/users/scratch/leo/scratch/converted_v11/long/0-20000-0-27612_CEUAS_merged_v1.nc


,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
18196505,1946-01-03 18:50:00,NaN,1,-2147483648,117,4903.325195
18196506,1946-01-03 18:50:00,NaN,1,-2147483648,117,9806.650391
18196507,1946-01-03 18:50:00,NaN,1,-2147483648,117,14709.974609
18196508,1946-01-03 18:50:00,NaN,1,-2147483648,117,19613.300781
18196509,1946-01-03 18:50:00,NaN,1,-2147483648,117,24516.625000
...,...,...,...,...,...,...
18313840,1951-04-10 16:40:00,NaN,1,-2147483648,117,196133.000000
18313841,1951-04-10 16:40:00,NaN,1,-2147483648,117,203586.046875
18314264,1951-04-16 04:30:00,NaN,1,-2147483648,117,19613.300781
18319995,1951-06-15 04:30:00,NaN,1,-2147483648,117,29419.949219


In [9]:
df[df.date_time == "1946-01-03 18:50:00"]

,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
18196492,1946-01-03 18:50:00,16800.0,1,-2147483648,117,118954.664062
18196493,1946-01-03 18:50:00,17100.0,1,-2147483648,117,117679.796875
18196494,1946-01-03 18:50:00,19500.0,1,-2147483648,117,110128.679688
18196495,1946-01-03 18:50:00,20200.0,1,-2147483648,117,107873.148438
18196496,1946-01-03 18:50:00,23200.0,1,-2147483648,117,100027.828125
18196497,1946-01-03 18:50:00,23900.0,1,-2147483648,117,98066.500000
18196498,1946-01-03 18:50:00,28100.0,1,-2147483648,117,88259.851562
18196499,1946-01-03 18:50:00,33000.0,1,-2147483648,117,78453.203125
18196500,1946-01-03 18:50:00,38500.0,1,-2147483648,117,68646.546875
18196501,1946-01-03 18:50:00,44600.0,1,-2147483648,117,58839.898438


##  0-20000-0-01107*

In [31]:
df[np.logical_and(df.date_time == "1925-04-14 09:00:00", df.observed_variable == 117)]

,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
158950,1925-04-14 09:00:00,1469.387695,0,-2147483648,117,14409.771484
158951,1925-04-14 09:00:00,2607.142822,0,-2147483648,117,25567.337891
158952,1925-04-14 09:00:00,270.408173,0,-2147483648,117,2651.798096
158953,1925-04-14 09:00:00,765.306152,0,-2147483648,117,7505.089355


In [32]:
df[np.logical_and(df.date_time == "1925-04-14 09:00:00", df.observed_variable == 140)]

,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
324293,1925-04-14 09:00:00,1469.387695,0,-2147483648,140,6.894400
324294,1925-04-14 09:00:00,2607.142822,0,-2147483648,140,9.396926
324295,1925-04-14 09:00:00,270.408173,0,-2147483648,140,8.863270
324296,1925-04-14 09:00:00,765.306152,0,-2147483648,140,7.878462


In [33]:
df

,date_time,z_coordinate,z_coordinate_type,z_coordinate_method,observed_variable,observation_value
0,1906-09-05 13:33:00,10750.0,0,-2147483648,106,2.200000e+01
1,1906-09-05 13:33:00,11450.0,0,-2147483648,106,3.500000e+02
2,1906-09-05 13:33:00,11900.0,0,-2147483648,106,2.720000e+02
3,1906-09-05 13:33:00,12300.0,0,-2147483648,106,3.360000e+02
4,1906-09-05 13:33:00,124.0,0,-2147483648,106,0.000000e+00
...,...,...,...,...,...,...
402548,1956-12-31 03:00:00,2100.0,0,-2147483648,140,-2.673109e+00
402549,1956-12-31 03:00:00,300.0,0,-2147483648,140,-9.444492e-07
402550,1956-12-31 03:00:00,3000.0,0,-2147483648,140,8.669847e+00
402551,1956-12-31 03:00:00,600.0,0,-2147483648,140,-6.746066e-07
